## Exploratory Analysis of Census Income Data
Predict whether income exceeds $50K/yr based on census data, also known as Adult dataset

### Source of information: 
* https://archive.ics.uci.edu/dataset/20/census+income



In [1]:
# Import required dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn.metrics import balanced_accuracy_score
warnings.filterwarnings("ignore")

import seaborn as sns

In [2]:
# Import the data


# Display the first 10 rows of the data
